In [1]:
from Technical import NSE
import pandas as pd
import requests
from datetime import datetime
from io import StringIO

In [2]:
nse = NSE()

In [3]:
# Some NSE basic functions this is available any NSE Object (Technical OR Derivatives)
# searching
search_res = nse.search('Syrma SGS')
print(search_res.get('symbols')[0].get('symbol'))

# Last Traded date
print(nse.get_last_traded_date())

# Last traded Status and Nifty val
print(nse.get_market_status_and_current_val())

# NSE Equity Meta info
print(nse.get_nse_equity_meta_info('SYRMA'))

SYRMA
2025-07-07
('Open', 25455.8)
{'symbol': 'SYRMA', 'companyName': 'Syrma SGS Technology Limited', 'industry': 'Industrial Products', 'activeSeries': ['EQ', 'T0'], 'debtSeries': [], 'isFNOSec': False, 'isCASec': False, 'isSLBSec': True, 'isDebtSec': False, 'isSuspended': False, 'tempSuspendedSeries': [], 'isETFSec': False, 'isDelisted': False, 'isin': 'INE0DYJ01015', 'slb_isin': 'INE0DYJ01015', 'listingDate': '2022-08-26', 'isMunicipalBond': False, 'isHybridSymbol': False, 'quotepreopenstatus': {'equityTime': '07-Jul-2025 14:10:36', 'preOpenTime': '07-Jul-2025 09:07:11', 'QuotePreOpenFlag': False}}


## New Way to get the OHLCV data from charting.nseindia.com (Lot more Timeframes and date ranges supported)

In [4]:
# NSE New OHLC data from charting.nseindia.com

# Get the mappings (Its not same as NSE website ticker, Get the mapping from `get_charting_mappings()` function)
charting_mappings = nse.get_charting_mappings() # get the Trading Symbol from this DataFrame (includes Equity and F&O instruments, Indices, etc.)
print(charting_mappings.head())

ohlc_df = nse.get_ohlc_from_charting('Nifty 500', '1Min', datetime(2025, 7, 4), datetime(2025, 7, 6))
print(ohlc_df.head())


   ScripCode     TradingSymbol       Description  InstrumentType
0      26000          Nifty 50          Nifty 50               0
1      26003         Nifty 500         Nifty 500               0
2      26008          Nifty IT          Nifty IT               0
3      26009        Nifty Bank        Nifty Bank               0
4      26010  Nifty Midcap 100  Nifty MIDCAP 100               0
            timestamp      open      high       low     close  volume
0 2025-07-04 09:15:59  23553.50  23567.30  23531.45  23545.15       1
1 2025-07-04 09:16:59  23544.85  23544.85  23521.85  23531.75       1
2 2025-07-04 09:17:59  23531.15  23546.60  23527.30  23540.75       1
3 2025-07-04 09:18:59  23541.20  23546.30  23527.85  23532.15       1
4 2025-07-04 09:19:59  23531.40  23533.90  23520.90  23524.80       1


In [5]:
imp_reports_link = nse.get_important_reports()
imp_reports_link[:5]

[{'name': 'CM-UDiFF Common Bhavcopy Final (zip)',
  'type': 'daily-reports',
  'category': 'capital-market',
  'section': 'equities',
  'link': 'https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20250704_F_0000.csv.zip'},
 {'name': 'CM - Market Activity Report',
  'type': 'daily-reports',
  'category': 'capital-market',
  'section': 'equities',
  'link': 'https://nsearchives.nseindia.com/archives/equities/mkt/MA040725.csv'},
 {'name': 'CM - Security-wise Delivery Positions',
  'type': 'daily-reports',
  'category': 'capital-market',
  'section': 'equities',
  'link': 'https://nsearchives.nseindia.com/archives/equities/mto/MTO_04072025.DAT'},
 {'name': 'CM - List of securities for auction',
  'type': 'daily-reports',
  'category': 'capital-market',
  'section': 'equities',
  'link': 'https://nsearchives.nseindia.com/content/nsccl/AUB_2025126_04072025.csv'},
 {'name': 'CM - Category-wise Turnover',
  'type': 'daily-reports',
  'category': 'capital-market',
  'section': 'e

In [6]:
# Let's say you need report of `CM - Security-wise Delivery Positions`
report_link = imp_reports_link[3].get('link')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/csv'
}
response = requests.get(report_link, headers=headers, timeout=15)
pd.read_csv(StringIO(response.text)).head()

,,,Confirmed list of securities to be auctioned in A 2025126 to be held on Jul 04 2025.
Sr No,Symbol,Series,Total Qty
1,360ONE,EQ,100
2,3PLAND,EQ,2
3,63MOONS,EQ,344
4,AARTIPHARM,EQ,1


In [7]:
# NSE turnover data
trade_df = nse.get_nse_turnover()
trade_df.head()

,segment,instrument,notionalTurnover,premiumTurnover,noOfTrades,volume,oivalue,mktTimeStamp,value,averageTrade,...,prevInstrument,prevNotionalTurnover,prevPremiumTurnover,prevNoOfTrades,prevVolume,prevOivalue,prevMktTimeStamp,prevValue,prevAverageTrade,prevNoOfOrders
0,CM,Equity,3.749791e+11,0.0,14136338,2376588844,0.0,2025-07-07 11:59:07,3.749791e+11,26526.0,...,None,8.853112e+11,0.0,29766554,3618100940,0.0,2025-07-04 16:00:00,8.853112e+11,NaN,567705087
1,CM,SME,1.849900e+09,0.0,9132,11430245,0.0,2025-07-07 11:59:07,1.849900e+09,202573.0,...,None,2.574248e+09,0.0,14488,16105824,0.0,2025-07-04 15:57:54,2.574248e+09,NaN,567705087
2,CM,Others,9.949460e+07,0.0,1475,198103,0.0,2025-07-07 11:59:06,9.949460e+07,67454.0,...,None,3.227739e+08,0.0,3338,1661101,0.0,2025-07-04 15:56:04,3.227739e+08,NaN,567705087
3,total,Total,NaN,NaN,14146945,2388217192,0.0,2025-07-07 11:59:07,3.769285e+11,26644.0,...,None,NaN,NaN,29784380,3635867865,0.0,2025-07-04 16:00:00,8.882082e+11,29821.0,567705087
4,FO,Index Futures,8.251432e+10,0.0,30564,42482,302001.0,2025-07-07 11:59:04,8.251432e+10,2699722.0,...,None,1.903632e+11,0.0,69750,98399,295952.0,2025-07-04 15:30:00,1.903632e+11,NaN,3821623965


## All Indices in NSE Exchange

In [4]:
nse.get_all_indices()

,key,index,indexSymbol,last,variation,percentChange,open,high,low,previousClose,...,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath,chartTodayPath,previousDay,oneWeekAgo,oneMonthAgo,oneYearAgo
0,INDICES ELIGIBLE IN DERIVATIVES,NIFTY 50,NIFTY 50,25457.30,-3.70,-0.01,25450.45,25489.80,25407.25,25461.00,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-50...,06-Jun-2025,1.83,https://nsearchives.nseindia.com/30d/NIFTY-50.svg,https://nsearchives.nseindia.com/today/NIFTY-5...,25405.30,25517.05,25003.05,24323.85
1,INDICES ELIGIBLE IN DERIVATIVES,NIFTY NEXT 50,NIFTY NEXT 50,68668.40,60.60,0.09,68692.85,68869.95,68544.60,68607.80,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-NE...,06-Jun-2025,0.90,https://nsearchives.nseindia.com/30d/NIFTY-NEX...,https://nsearchives.nseindia.com/today/NIFTY-N...,68355.85,68998.20,67992.85,73426.55
2,INDICES ELIGIBLE IN DERIVATIVES,NIFTY BANK,NIFTY BANK,56985.15,-46.75,-0.08,56938.70,57152.20,56838.55,57031.90,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-BA...,06-Jun-2025,0.80,https://nsearchives.nseindia.com/30d/NIFTY-BAN...,https://nsearchives.nseindia.com/today/NIFTY-B...,56791.95,57312.75,56578.40,52660.35
3,INDICES ELIGIBLE IN DERIVATIVES,NIFTY FINANCIAL SERVICES,NIFTY FIN SERVICE,26843.85,-22.45,-0.08,26837.85,26894.70,26772.05,26866.30,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-FI...,06-Jun-2025,0.06,https://nsearchives.nseindia.com/30d/NIFTY-FIN...,https://nsearchives.nseindia.com/today/NIFTY-F...,26734.90,27174.45,26848.90,23641.75
4,INDICES ELIGIBLE IN DERIVATIVES,NIFTY MIDCAP SELECT,NIFTY MID SELECT,13398.80,-17.20,-0.13,13425.95,13462.05,13352.40,13416.00,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-MI...,06-Jun-2025,2.05,https://nsearchives.nseindia.com/30d/NIFTY-MID...,https://nsearchives.nseindia.com/today/NIFTY-M...,13462.55,13433.85,13146.00,12520.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,FIXED INCOME INDICES,NIFTY COMPOSITE G-SEC INDEX,NIFTY GS COMPSITE,3028.13,1.18,0.03,3027.51,3029.12,3027.45,3026.95,...,05-Jul-2024,https://nsearchives.nseindia.com/365d/NIFTY-GS...,06-Jun-2025,-0.28,https://nsearchives.nseindia.com/30d/NIFTY-GS-...,https://nsearchives.nseindia.com/today/NIFTY-G...,3025.35,3016.03,3034.25,2734.96
119,FIXED INCOME INDICES,NIFTY BHARAT BOND INDEX - APRIL 2030,BHARATBOND-APR30,1532.95,0.72,0.01,1533.07,1533.07,1532.95,1532.23,...,-,-,06-Jun-2025,-0.41,https://nsearchives.nseindia.com/30d/BHARATBON...,https://nsearchives.nseindia.com/today/BHARATB...,1532.74,1525.52,1538.61,NaN
120,FIXED INCOME INDICES,NIFTY BHARAT BOND INDEX - APRIL 2031,BHARATBOND-APR31,1398.01,0.61,0.00,1398.16,1398.16,1397.99,1397.40,...,-,-,06-Jun-2025,-0.55,https://nsearchives.nseindia.com/30d/BHARATBON...,https://nsearchives.nseindia.com/today/BHARATB...,1397.33,1391.81,1405.10,NaN
121,FIXED INCOME INDICES,NIFTY BHARAT BOND INDEX - APRIL 2032,BHARATBOND-APR32,1272.86,0.69,0.01,1272.87,1272.87,1272.86,1272.17,...,-,-,06-Jun-2025,-0.70,https://nsearchives.nseindia.com/30d/BHARATBON...,https://nsearchives.nseindia.com/today/BHARATB...,1271.78,1269.69,1281.19,NaN


### Get All Equities from an index

In [5]:
# default it takes `SECURITIES IN F&O`
# This supports all options in this drop down` https://www.nseindia.com/market-data/live-equity-market`
index_df = nse.get_equities_data_from_index()
index_df.head()

index_df = nse.get_equities_data_from_index(index='NIFTY 500')
index_df.head()

,priority,symbol,identifier,open,dayHigh,dayLow,lastPrice,previousClose,change,pChange,...,perChange30d,series,meta_symbol,meta_companyName,meta_industry,meta_slb_isin,meta_listingDate,meta_quotepreopenstatus_equityTime,meta_quotepreopenstatus_preOpenTime,meta_quotepreopenstatus_QuotePreOpenFlag
0,1,NIFTY 500,NIFTY 500,23561.6,23595.30,23506.50,23540.95,23562.45,-21.50,-0.09,...,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,JPPOWER,JPPOWEREQN,19.1,22.49,18.95,22.36,18.95,3.41,17.99,...,16.69,EQ,JPPOWER,Jaiprakash Power Ventures Limited,Power Generation,INE351F01018,2005-04-18,07-Jul-2025 14:10:57,07-Jul-2025 09:07:11,False
2,0,GODREJCP,GODREJCPEQN,1225.2,1271.90,1225.20,1264.90,1192.90,72.00,6.04,...,-1.42,EQ,GODREJCP,Godrej Consumer Products Limited,Personal Care,INE102D01028,2001-06-20,07-Jul-2025 14:10:57,07-Jul-2025 09:07:11,False
3,0,ALKYLAMINE,ALKYLAMINEEQN,2295.2,2438.80,2286.60,2401.90,2295.20,106.70,4.65,...,15.83,EQ,ALKYLAMINE,Alkyl Amines Chemicals Limited,Specialty Chemicals,INE150B01039,2007-11-26,07-Jul-2025 14:10:57,07-Jul-2025 09:07:11,False
4,0,JYOTHYLAB,JYOTHYLABEQN,342.0,364.80,341.00,356.60,342.00,14.60,4.27,...,0.96,EQ,JYOTHYLAB,Jyothy Labs Limited,Household Products,INE668F01031,2007-12-19,07-Jul-2025 14:10:57,07-Jul-2025 09:07:11,False


In [9]:
index_df = nse.get_equities_data_from_index(index='NIFTY 500')

In [13]:
# OHLC Data
nse.get_ohlc_data('JPPOWER', is_index=False)

,open,high,low,close
timestamp,,,,
2025-07-07 09:00:00,18.95,19.13,18.95,19.05
2025-07-07 09:05:00,19.05,19.13,19.05,19.10
2025-07-07 09:10:00,NaN,NaN,NaN,NaN
2025-07-07 09:15:00,19.10,19.36,18.99,19.34
2025-07-07 09:20:00,19.32,20.36,19.30,20.35
...,...,...,...,...
2025-07-07 15:35:00,NaN,NaN,NaN,NaN
2025-07-07 15:40:00,22.65,22.65,22.65,22.65
2025-07-07 15:45:00,22.65,22.65,22.65,22.65


## Trade Info Data

In [12]:
nse.get_trade_info('CENTRALBK').T

,0
currentMarketType,NM
noBlockDeals,True
bulkBlockDeals,"[{'name': 'Session I'}, {'name': 'Session II'}]"
info_symbol,CENTRALBK
info_companyName,Central Bank of India
...,...
securityWiseDP_quantityTraded,3897861
securityWiseDP_deliveryQuantity,1526650
securityWiseDP_deliveryToTradedQuantity,39.17
securityWiseDP_seriesRemarks,None


## Corporate Disclosures

In [12]:
nse.get_corporate_disclosures('EASEMYTRIP')

{'EASEMYTRIP': {'latest_announcements': {'data': [{'symbol': 'EASEMYTRIP',
     'broadcastdate': '02-Jul-2025 12:21:50',
     'subject': 'Change in Directors/ Key Managerial Personnel/ Auditor/ Compliance Officer/ Share Transfer Agent'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '01-Jul-2025 20:09:27',
     'subject': 'Change in Director(s)'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '01-Jul-2025 19:50:47',
     'subject': 'Shareholders meeting'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '01-Jul-2025 18:29:01',
     'subject': 'Cessation'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '30-Jun-2025 19:38:37',
     'subject': 'Clarification - Financial Results'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '27-Jun-2025 16:06:56',
     'subject': 'Trading Window-XBRL'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '25-Jun-2025 19:07:04',
     'subject': 'General Updates'},
    {'symbol': 'EASEMYTRIP',
     'broadcastdate': '24-Jun-2025 12:17

## SME Data

In [13]:
# Get SME (Small Medium Enterprises) stocks data
nse.get_sme_stocks()

,symbol,series,open,dayHigh,dayLow,lastPrice,change,pChange,previousClose,stockIndClosePrice,...,nearWKH,nearWKL,ca,chartTodayPath,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath
0,IDEALTECHO,SM,155.00,155.00,155.00,155.00,23.00,17.42,146.00,0,...,0.000000,-76.136364,,https://nsearchives.nseindia.com/today/IDEALTE...,-,-,-,04-Jun-2025,5.6,https://nsearchives.nseindia.com/30d/IDEALTECH...
1,JAINIK,SM,67.50,75.20,67.50,74.25,6.40,9.43,61.20,0,...,13.763066,-22.524752,,https://nsearchives.nseindia.com/today/JAINIKS...,-,-,-,-,-,-
2,QMSMEDI,SM,81.90,85.00,80.85,85.00,7.25,9.32,79.15,0,...,44.769331,-17.484451,,https://nsearchives.nseindia.com/today/QMSMEDI...,-47.34,05-Jul-2024,https://nsearchives.nseindia.com/365d/QMSMEDI-...,06-Jun-2025,-12.15,https://nsearchives.nseindia.com/30d/QMSMEDI-S...
3,MONOLITH,SM,319.95,345.00,310.00,337.40,27.80,8.98,321.50,0,...,2.202899,-53.748006,,https://nsearchives.nseindia.com/today/MONOLIT...,-,-,-,-,-,-
4,PATILAUTOM,SM,204.90,231.65,202.00,219.00,14.75,7.22,194.55,0,...,5.460825,-41.290323,,https://nsearchives.nseindia.com/today/PATILAU...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,TCL,SM,114.95,114.95,108.50,108.60,-6.40,-5.57,119.00,0,...,56.262586,-40.764744,,https://nsearchives.nseindia.com/today/TCLSMN.svg,-47.87,05-Jul-2024,https://nsearchives.nseindia.com/365d/TCL-SM.svg,06-Jun-2025,25.96,https://nsearchives.nseindia.com/30d/TCL-SM.svg
425,USASEEDS,SM,163.00,164.00,157.00,160.00,-10.00,-5.88,176.90,0,...,52.920406,-22.417751,,https://nsearchives.nseindia.com/today/USASEED...,-44.54,05-Jul-2024,https://nsearchives.nseindia.com/365d/USASEEDS...,05-Jun-2025,-17.07,https://nsearchives.nseindia.com/30d/USASEEDS-...
426,NEPHROCARE,SM,114.15,117.25,99.95,110.75,-7.70,-6.50,123.15,0,...,70.762936,-10.805403,,https://nsearchives.nseindia.com/today/NEPHROC...,-,-,-,06-Jun-2025,-10.64,https://nsearchives.nseindia.com/30d/NEPHROCAR...
427,MICROPRO,SM,24.40,24.40,21.55,22.20,-1.55,-6.53,24.50,0,...,55.060729,-22.991690,,https://nsearchives.nseindia.com/today/MICROPR...,-46.33,05-Jul-2024,https://nsearchives.nseindia.com/365d/MICROPRO...,06-Jun-2025,1.06,https://nsearchives.nseindia.com/30d/MICROPRO-...


## SGB Data

In [14]:
# Get SGB (Sovereign Gold Bond) data
nse.get_sgb_data()

,symbol,open,high,low,issue_price,ltP,chn,per,qty,trdVal,...,nearWKH,nearWKL,maturityDate,chartTodayPath,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath
0,SGBNOV25,10397.9,10397.9,10397.9,2884,10397.9,567.9,5.78,5,51989.5,...,0.000000,-44.918467,NOV25,https://nsearchives.nseindia.com/today/SGBNOV2...,29.36,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBNOV25...,02-Jun-2025,5.59,https://nsearchives.nseindia.com/30d/SGBNOV25-...
1,SGBNOV26,9919.99,9919.99,9919.99,3133,9919.99,150.98,1.55,1,9919.99,...,0.800100,-39.718169,NOV26,https://nsearchives.nseindia.com/today/SGBNOV2...,27.88,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBNOV26...,06-Jun-2025,1.5,https://nsearchives.nseindia.com/30d/SGBNOV26-...
2,SGBFEB28IX,9780,9880,9779.99,0,9880,100,1.02,3,29439.99,...,0.703518,-37.988827,B28IX,https://nsearchives.nseindia.com/today/SGBFEB2...,30.21,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBFEB28...,05-Jun-2025,0.82,https://nsearchives.nseindia.com/30d/SGBFEB28I...
3,SGBJAN30IX,9975.59,9999.99,9830.02,0,9999.99,74.03,0.75,16,159197.12,...,1.910106,-40.844930,N30IX,https://nsearchives.nseindia.com/today/SGBJAN3...,30.60,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBJAN30...,06-Jun-2025,2.07,https://nsearchives.nseindia.com/30d/SGBJAN30I...
4,SGBAUG30,9926,10289.99,9926,0,10084.99,58.49,0.58,21,210912.45,...,3.019617,-35.806491,AUG30,https://nsearchives.nseindia.com/today/SGBAUG3...,31.11,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBAUG30...,06-Jun-2025,0.77,https://nsearchives.nseindia.com/30d/SGBAUG30-...
5,SGBFEB29XI,9830.01,9866,9821,0,9866,35.99,0.37,26,255819.98,...,1.586035,-35.150685,B29XI,https://nsearchives.nseindia.com/today/SGBFEB2...,30.79,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBFEB29...,06-Jun-2025,1.36,https://nsearchives.nseindia.com/30d/SGBFEB29X...
6,SGBSEP31II,10001.1,10330,10000,0,10114.99,31.07,0.31,1321,13290197.91,...,3.436850,-34.667226,P31II,https://nsearchives.nseindia.com/today/SGBSEP3...,30.86,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBSEP31...,06-Jun-2025,1.4,https://nsearchives.nseindia.com/30d/SGBSEP31I...
7,SGBOCT27VI,9775,9799,9775,0,9799,24,0.25,15,146745,...,1.517588,-36.097222,T27VI,https://nsearchives.nseindia.com/today/SGBOCT2...,30.17,04-Jul-2024,https://nsearchives.nseindia.com/365d/SGBOCT27...,06-Jun-2025,1.31,https://nsearchives.nseindia.com/30d/SGBOCT27V...
8,SGBAPR28I,9874,9950,9800,0,9850,24.06,0.24,126,1242419.22,...,1.735540,-36.596866,PR28I,https://nsearchives.nseindia.com/today/SGBAPR2...,26.49,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBAPR28...,06-Jun-2025,0.57,https://nsearchives.nseindia.com/30d/SGBAPR28I...
9,SGBMAY29I,9850,9850,9800.01,0,9844.99,7.29,0.07,95,935317.75,...,1.055377,-35.196237,AY29I,https://nsearchives.nseindia.com/today/SGBMAY2...,30.68,05-Jul-2024,https://nsearchives.nseindia.com/365d/SGBMAY29...,06-Jun-2025,1.96,https://nsearchives.nseindia.com/30d/SGBMAY29I...


## ETF Data

In [15]:
# Get ETF (Exchange Traded Fund) data
nse.get_all_etf()

,symbol,assets,open,high,low,ltP,chn,per,qty,trdVal,...,stockIndClosePrice,nearWKH,nearWKL,chartTodayPath,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath
0,MASPTOP50,S&P 500 Top 50 Total Return Index,57.55,57.55,56.15,57.02,0.87,1.55,245410,13968737.2,...,0,12.209392,-56.304825,https://nsearchives.nseindia.com/today/MASPTOP...,27.27,05-Jul-2024,https://nsearchives.nseindia.com/365d/MASPTOP5...,06-Jun-2025,3.71,https://nsearchives.nseindia.com/30d/MASPTOP50...
1,MAFANG,NYSE FANG+ Total Return Index,152.28,152.28,150.18,152.28,2.1,1.40,182842,27819410.3,...,0,0.781861,-70.621849,https://nsearchives.nseindia.com/today/MAFANGE...,52.13,05-Jul-2024,https://nsearchives.nseindia.com/365d/MAFANG-E...,06-Jun-2025,10.74,https://nsearchives.nseindia.com/30d/MAFANG-EQ...
2,FMCGIETF,Nifty FMCG Index,58.17,59.22,56.94,59.1,0.69,1.18,1334771,78631359.61,...,0,16.994382,-11.509434,https://nsearchives.nseindia.com/today/FMCGIET...,-3.15,05-Jul-2024,https://nsearchives.nseindia.com/365d/FMCGIETF...,06-Jun-2025,-1.63,https://nsearchives.nseindia.com/30d/FMCGIETF-...
3,IDFNIFTYET,Nifty 50,279.27,281.43,277.24,279.87,1.98,0.71,832,232727.04,...,0,6.347878,-19.088549,https://nsearchives.nseindia.com/today/IDFNIFT...,6.01,05-Jul-2024,https://nsearchives.nseindia.com/365d/IDFNIFTY...,06-Jun-2025,2.36,https://nsearchives.nseindia.com/30d/IDFNIFTYE...
4,BANKPSU,Nifty PSU Bank Total Return Index,73.79,73.79,71.52,72.12,0.48,0.67,4101,294943.92,...,0,6.337662,-30.439501,https://nsearchives.nseindia.com/today/BANKPSU...,-,-,-,06-Jun-2025,0.28,https://nsearchives.nseindia.com/30d/BANKPSU-E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,BSLGOLDETF,Gold,88.38,88.38,85.25,85.54,-0.69,-0.80,24121,2063069.13,...,0,8.051166,-51.210889,https://nsearchives.nseindia.com/today/BSLGOLD...,32.72,05-Jul-2024,https://nsearchives.nseindia.com/365d/BSLGOLDE...,06-Jun-2025,0.22,https://nsearchives.nseindia.com/30d/BSLGOLDET...
256,GROWWDEFNC,Nifty India Defence Index- Total Return Index,90.35,91.69,88.17,88.7,-0.76,-0.85,190259,16874070.71,...,0,5.527745,-78.005218,https://nsearchives.nseindia.com/today/GROWWDE...,-,-,-,06-Jun-2025,0.1,https://nsearchives.nseindia.com/30d/GROWWDEFN...
257,UNIONGOLD,Commodity Gold,95.7,95.7,95.2,95.2,-0.85,-0.88,567,54148.5,...,0,4.800000,-16.097561,https://nsearchives.nseindia.com/today/UNIONGO...,-,-,-,06-Jun-2025,0.05,https://nsearchives.nseindia.com/30d/UNIONGOLD...
258,MOGSEC,Nifty 5 yr Benchmark G-Sec Index,62.9,62.9,61.64,62.31,-0.62,-0.99,7752,483182.16,...,0,5.303951,-12.493230,https://nsearchives.nseindia.com/today/MOGSECE...,11.4,05-Jul-2024,https://nsearchives.nseindia.com/365d/MOGSEC-E...,06-Jun-2025,1.5,https://nsearchives.nseindia.com/30d/MOGSEC-EQ...


## ALL Block Deals

In [16]:
# Get all block deals happened today
nse.get_all_today_block_deals()

""


# India VIX

In [17]:
# India (Bharat) VIX (Volatility Index) data
nse.get_india_vix(interval='3Min')

INDIA VIX is No Longer Available on NSE India :( 
Please Use INDIA VIX from MONEYCONTROL

        from Fundamentals import MoneyControl
        moneycontrol = MoneyControl()
        moneycontrol.get_india_vix(interval='1') # Only `1d` (for day interval) or `1`  (for minute interval) is supported
        


""
